# Parallel Computing Lab 3: Parallelized Breadth-First Search
November 18, 2022
- Chad Spector
- Eric Chao

## Dependencies: Compare nodeOutput, Compare nextLevelNodes

### `compareNextLevelNodes.cu`

In [ ]:
%%writefile compareNextLevelNodes.cu
#include <stdio.h>
#include <stdlib.h>
void sort(int *pointer, int size){
  //get from https://stackoverflow.com/questions/13012594/sorting-with-pointers-instead-of-indexes
    int *i, *j, temp;
    for(i = pointer; i < pointer + size; i++){
        for(j = i + 1; j < pointer + size; j++){
            if(*j < *i){
                temp = *j;
                *j = *i;
                *i = temp;
            }
        }
    }
}

void compareNextLevelNodeFiles(char *file_name1, char *file_name2) 
{ 

  
    FILE* fp_1 = fopen(file_name1, "r");
    if (fp_1 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 

    FILE* fp_2 = fopen(file_name2, "r");
    if (fp_2 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len_1;
    int len_2;
    int length_file_1 = fscanf(fp_1, "%d", &len_1);
    int length_file_2 = fscanf(fp_2, "%d", &len_2);

    if(length_file_1 != length_file_2){
      fprintf(stderr, "Wrong file lengths: f1 %d f2 %d\n", len_1, len_2);
      exit(1);
    }
    int *input1 = (int *)malloc(len_1 * sizeof(int));
    int *input2 = (int *)malloc(len_2 * sizeof(int));




    int temp1;
    int temp2;

    while ((fscanf(fp_1, "%d", &temp1) == 1) && (fscanf(fp_2, "%d", &temp2) == 1)) {
        (input1)[counter] = temp1;
        (input2)[counter] = temp2;
        counter++;
    }

    sort(input1, len_1);
    sort(input2, len_2);

    for(int i=0; i< len_1; i++){
      if(input1[i] != input2[i]){
        fprintf(stderr, "Something goes wrong\n");
        exit(1);
      }
    }

    fprintf(stderr, "No errors!\n");
        exit(1);

} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareNextLevelNodeFiles(argv[1], argv[2]);
}


Writing compareNextLevelNodes.cu


In [ ]:
!nvcc compareNextLevelNodes.cu -o compareNextLevelNodes

### `compareNodeOutput.cu`

In [ ]:
%%writefile compareNodeOutput.cu
#include <stdio.h>
#include <stdlib.h>


void compareFiles(char *file_name1, char *file_name2) 
{ 
//get from https://www.tutorialspoint.com/c-program-to-compare-two-files-and-report-mismatches
FILE* fp1 = fopen(file_name1, "r");
FILE* fp2 = fopen(file_name2, "r");
    // fetching character of two file 
    // in two variable ch1 and ch2 
    char ch1 = getc(fp1); 
    char ch2 = getc(fp2); 
  
    // error keeps track of number of errors 
    // pos keeps track of position of errors 
    // line keeps track of error line 
    int error = 0, pos = 0, line = 1; 
  
    // iterate loop till end of file 
    while (ch1 != EOF && ch2 != EOF) 
    { 
        pos++; 
  
        // if both variable encounters new 
        // line then line variable is incremented 
        // and pos variable is set to 0 
        if (ch1 == '\n' && ch2 == '\n') 
        { 
            line++; 
            pos = 0; 
        } 
  
        // if fetched data is not equal then 
        // error is incremented 
        if (ch1 != ch2) 
        { 
            error++; 
            printf("Line Number : %d \tError"
               " Position : %d \n", line, pos); 
        } 
  
        // fetching character until end of file 
        ch1 = getc(fp1); 
        ch2 = getc(fp2); 
    } 
  
    printf("Total Errors : %d\t", error); 
} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareFiles(argv[1], argv[2]);
}


Writing compareNodeOutput.cu


In [ ]:
!nvcc compareNodeOutput.cu -o compareNodeOutput

# Sequential BFS

### sequential.cu

In [ ]:
%%writefile sequential.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

#define MAX_QUEUE_SIZE 41000


int gate_solver(int gate, int input1, int input2) {
    int output;
    switch (gate) {
            case AND:
                output = input1 && input2;
                break;
            case OR:
                output = input1 || input2;
                break;
            case NAND:
                output = !(input1 && input2);
                break;
            case NOR:
                output = !(input1 || input2);
                break;
            case XOR:
                output = (input1 || input2) && !(input1 && input2);
                break;
            case XNOR:
                output = (input1 && input2) || (!input1 && !input2);
                break;
    }
    return output;
}

/**
 * @brief Reads an input file containing length L in first line and one integer for each of the next L lines, storing the values in the integer array provided.
 * 
 * @param input1 pointer to an integer array to store the data
 * @param filepath character array, name of input file to open
 * @return int, length of resulting array
 */
int read_input_one_two_four(int **input1, char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    
    // Allocate len entries and assign the pointer to out input reference
    *input1 = ( int *)malloc(len * sizeof(int));
    int temp1;
    
    // read line by line, one integer per line
    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;

}

/**
 * @brief Reads a CSV formatted file with information for every node gate.
 * 
 * @param input1 Pointer to Visited array
 * @param input2 Pointer to Logic Gate types array
 * @param input3 Pointer to Logic Gate inputs array
 * @param input4 Pointer to Logic Gate outputs array
 * @param filepath 
 * @return int, the number of node entries
 */
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));    // visited[i], either 0 or 1
    *input2 = ( int *)malloc(len * sizeof(int));    // gate[i], see macros definitions
    *input3 = ( int *)malloc(len * sizeof(int));    // input[i], fixed input to node i
    *input4 = ( int *)malloc(len * sizeof(int));    // output[i], -1 if not computed

    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len; // number of total nodes in circuit
}

void BFS(int* currLevelNodes, int numCurrLevelNodes, int* nodePtrs, int* nodeNeighbors, int* nodeVisited, int* nodeGate, int* nodeInput, int* nodeOutput, int* nextLevelNodes, int* numNextLevelNodes) {
    // Traverse current level queue
    for (int idx = 0; idx < numCurrLevelNodes; idx++) { // while idx < numCurrLevelNodes
        int node = currLevelNodes[idx];
        // Loop over all neighbors of the node
        // printf("Nodeptrs+1 %d \n", nodePtrs[node + 1]);
        for (int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) {
            int neighbor = nodeNeighbors[nbrIdx];
            // If neighbor hasn't been visited yet
            if (!nodeVisited[neighbor]) {
                // Mark it as visited and add it to queue
                nodeVisited[neighbor] = 1;
                nodeOutput[neighbor] = gate_solver(nodeGate[neighbor], nodeInput[neighbor], nodeOutput[node]);
                // printf("nextLevelNodes LEN %d index %d", numNextLevelNodes);
                nextLevelNodes[*numNextLevelNodes] = neighbor;
                ++(*numNextLevelNodes);
            }
        }
    }
}

int main(int argc, char *argv[]) {
    if (argc != 7)
    {
        printf("\nIncorrect number of arguments. Please run the program as follows:\n");
        printf("./sequential <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    }
    
    // Read input and output file names from command
    char* input_1 = argv[1]; 
    char* input_2 = argv[2]; 
    char* input_3 = argv[3]; 
    char* input_4 = argv[4];
    char* output_nodeOutput = argv[5]; 
    char* output_nextLevelNodes = argv[6];
    
    // Variables
    int numNodePtrs;          
    int numNodes;             
    int *nodePtrs_h;          
    int *nodeNeighbors_h;      
    int *nodeVisited_h;   
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes;
    int numNextLevelNodes_h;
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;

    int *nextLevelNodes_h; // to write in output file
    nextLevelNodes_h = ( int *)malloc(MAX_QUEUE_SIZE * sizeof(int));
    
    numNextLevelNodes_h = 0;

    numNodePtrs = read_input_one_two_four(&nodePtrs_h, input_1);

    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, input_2);

    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);

    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);
    // Measure runtime of sequential BFS
    clock_t startTime = clock();
    BFS(currLevelNodes_h, numCurrLevelNodes, nodePtrs_h, nodeNeighbors_h, nodeVisited_h, nodeGate_h, nodeInput_h, nodeOutput_h, nextLevelNodes_h, &numNextLevelNodes_h);
    clock_t stopTime = clock();

    // CLOCKS_PER_SEC is the number of clock tics elapsed in a second, 
    // which gives us the precise CPU time consumed by a task
    double totalTimeInMs = (((double)stopTime - (double)startTime) * 1000.0) / CLOCKS_PER_SEC;
    // print total time elapsed for sequential gate simulation
    printf("Sequential Simultation: %f ms\n", totalTimeInMs);

    FILE* output_file_node = fopen(output_nodeOutput, "w"); 
    FILE* output_file_nextLevelNode = fopen(output_nextLevelNodes, "w");

    // Print out the results
    fprintf(output_file_node, "%d\n", numNodes);
    for (int i = 0; i < numNodes; i++) { 
        fprintf(output_file_node, "%d\n", nodeOutput_h[i]); 
    }
    fclose(output_file_node);

    fprintf(output_file_nextLevelNode, "%d\n", numNextLevelNodes_h);
    for (int i = 0; i < numNextLevelNodes_h; i++) { 
        fprintf(output_file_nextLevelNode, "%d\n", nextLevelNodes_h[i]); 
    }
    fclose(output_file_nextLevelNode);
}


Writing sequential.cu


### Compile and Run

In [ ]:
!nvcc sequential.cu -o sequential

sequential.cu(152): warning: variable "numNodePtrs" was set but never used

sequential.cu(157): warning: variable "numTotalNeighbors_h" was set but never used



In [ ]:
!./sequential input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

Sequential Simultation: 2.272000 ms


### Verify Results

In [ ]:
!./compareNextLevelNodes sol_nextLevelNodes.raw nextLevelNodes.raw

No errors!


In [ ]:
!./compareNodeOutput sol_nodeOutput.raw nodeOutput.raw

Total Errors : 0	

# Parallel BFS (global queue)

In [ ]:
%%writefile global_queuing.cu
#include <stdio.h>
#include <stdlib.h>
// CUDA runtime
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

#define MAX_QUEUE_SIZE 200000
// #define BLOCK_SIZE 32 // blockSize (32, 64, 128)
// #define NUM_BLOCKS 10 // numBlock (10, 25, 35)

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

__device__ int numNextLevelNodes_d = 0;

// Macro for Cuda Debugging
#define gpuErrorCheck(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}


__device__ int gate_solver(int gate, int input1, int input2) {
    int output;
    switch (gate) {
            case AND:
                output = input1 && input2;
                break;
            case OR:
                output = input1 || input2;
                break;
            case NAND:
                output = !(input1 && input2);
                break;
            case NOR:
                output = !(input1 || input2);
                break;
            case XOR:
                output = (input1 || input2) && !(input1 && input2);
                break;
            case XNOR:
                output = (input1 && input2) || (!input1 && !input2);
                break;
    }
    return output;
}

/**
 * @brief Reads an input file containing length L in first line and one integer for each of the next L lines, storing the values in the integer array provided.
 * 
 * @param input1 pointer to an integer array to store the data
 * @param filepath character array, name of input file to open
 * @return int, length of resulting array
 */
int read_input_one_two_four(int **input1, char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    
    // Allocate len entries and assign the pointer to out input reference
    *input1 = ( int *)malloc(len * sizeof(int));
    int temp1;
    
    // read line by line, one integer per line
    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;

}

/**
 * @brief Reads a CSV formatted file with information for every node gate.
 * 
 * @param input1 Pointer to Visited array
 * @param input2 Pointer to Logic Gate types array
 * @param input3 Pointer to Logic Gate inputs array
 * @param input4 Pointer to Logic Gate outputs array
 * @param filepath 
 * @return int, the number of node entries
 */
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));    // visited[i], either 0 or 1
    *input2 = ( int *)malloc(len * sizeof(int));    // gate[i], see macros definitions
    *input3 = ( int *)malloc(len * sizeof(int));    // input[i], fixed input to node i
    *input4 = ( int *)malloc(len * sizeof(int));    // output[i], -1 if not computed

    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len; // number of total nodes in circuit
}


__global__ void globalQueueBFS(int* currLevelNodes, int numCurrLevelNodes, int* nodePtrs, int* nodeNeighbors, int* nodeVisited, int* nodeGate, int* nodeInput, int* nodeOutput, int* nextLevelNodes, int nodes_per_thread) {
    
    const unsigned int tid = (blockIdx.x * blockDim.x) + threadIdx.x;

    const unsigned int start = tid * nodes_per_thread;
    const int thread_limit = start + nodes_per_thread;
    const bool debug = tid == 0; 

    for (unsigned int idx = start; idx < thread_limit && idx < numCurrLevelNodes; idx++) {
        // get the node
        unsigned int node = currLevelNodes[idx];

        // Loop through neighbors
        for (unsigned int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) {
            int neighbor = nodeNeighbors[nbrIdx];
            // If neighbor hasn't been visited yet
            int already_visited = atomicExch(&(nodeVisited[neighbor]), 1);
            if (!already_visited) {
                int output = gate_solver(nodeGate[neighbor], nodeInput[neighbor], nodeOutput[node]);
                nodeOutput[neighbor] = output;
                // Atomically add to queue
                int new_i = atomicAdd(&numNextLevelNodes_d, 1);
                nextLevelNodes[new_i] = neighbor;

                // if (debug) {
                //     printf("Result nextQueue[%d]=%d\n", new_i, nextLevelNodes[new_i]);
                // }
            }
        }
    }
    // printf("Thread %d finished processing queue %d-%d\n", tid, start, start + thread_limit);

}


int main(int argc, char *argv[]) {
    // Variables
    if (argc != 9)
    {
        printf("\nIncorrect number of arguments. Please run the program as follows:\n");
        printf("./global_queuing <numBlock> <blockSize> <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    }
    // Read input and output file names from command
    int numBlock;
    int blockSize; 
    sscanf(argv[1], "%d", &numBlock);
    sscanf(argv[2], "%d", &blockSize);

    char* input_1 = argv[3]; 
    char* input_2 = argv[4]; 
    char* input_3 = argv[5];
    char* input_4 = argv[6];
    char* output_nodeOutput = argv[7]; 
    char* output_nextLevelNodes = argv[8];
    
    // Variables
    int numNodePtrs;          
    int numNodes;             
    int *nodePtrs_h;          
    int *nodeNeighbors_h;      
    int *nodeVisited_h;   
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes;
    int numNextLevelNodes_h;
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;
    
    numNextLevelNodes_h = 0;
    numNodePtrs = read_input_one_two_four(&nodePtrs_h, input_1);
    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, input_2);
    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);
    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);
    
    // Allocate GPU Memory 

    int *nodePtrs_d;          
    int *nodeNeighbors_d;      
    int *nodeVisited_d;   
    int numTotalNeighbors_d;
    int *currLevelNodes_d;
    int *nodeGate_d;
    int *nodeInput_d;
    int *nodeOutput_d;

    int* nextLevelNodes_d;
    gpuErrorCheck(cudaMalloc((void **)&nextLevelNodes_d, MAX_QUEUE_SIZE * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeVisited_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodePtrs_d, numNodePtrs * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeNeighbors_d, numTotalNeighbors_h * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&currLevelNodes_d, numCurrLevelNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeGate_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeInput_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeOutput_d, numNodes * sizeof(int)));

    gpuErrorCheck(cudaMemcpy(nodeVisited_d, nodeVisited_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodePtrs_d, nodePtrs_h, numNodePtrs * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeNeighbors_d, nodeNeighbors_h, numTotalNeighbors_h * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(currLevelNodes_d, currLevelNodes_h, numCurrLevelNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeGate_d, nodeGate_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeInput_d, nodeInput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeOutput_d, nodeOutput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));


    // Start timer
    float memsettime;
    cudaEvent_t start, stop;
    gpuErrorCheck(cudaEventCreate(&start));
    gpuErrorCheck(cudaEventCreate(&stop));
    gpuErrorCheck(cudaEventRecord(start, 0));

    // CALL THE KERNEL
    printf("%d Blocks, %d Threads/Block\n", numBlock, blockSize);

    int capacity = numBlock * blockSize;
    int remainder = (int)((numCurrLevelNodes % capacity) > 0);
    int nodes_per_thread = (numCurrLevelNodes / capacity) + remainder;
    printf("%d Nodes per Thread\n", nodes_per_thread);
    globalQueueBFS<<<numBlock, blockSize>>>(currLevelNodes_d, numCurrLevelNodes, nodePtrs_d, nodeNeighbors_d, nodeVisited_d, nodeGate_d, nodeInput_d, nodeOutput_d, nextLevelNodes_d, nodes_per_thread);
    cudaDeviceSynchronize();

    // Stop timer
    gpuErrorCheck(cudaEventRecord(stop, 0));
    gpuErrorCheck(cudaEventSynchronize(stop));
    gpuErrorCheck(cudaEventElapsedTime(&memsettime, start, stop));
    // print total time elapsed for global queuing gate simulation
    printf("*** Global Queuing Execution Time: %f ms***\n", memsettime);

    gpuErrorCheck(cudaEventDestroy(start));
    gpuErrorCheck(cudaEventDestroy(stop));

    // Cleanup;
    cudaFree(nodePtrs_d);
    free(nodePtrs_h);
    cudaFree(nodeNeighbors_d);
    free(nodeNeighbors_h);
    cudaFree(nodeVisited_d);
    free(nodeVisited_h);
    cudaFree(currLevelNodes_d);
    free(currLevelNodes_h);
    cudaFree(nodeGate_d);
    free(nodeGate_h);
    cudaFree(nodeInput_d);
    free(nodeInput_h);

    // Copy results
    gpuErrorCheck(cudaMemcpyFromSymbol(&numNextLevelNodes_h, numNextLevelNodes_d, sizeof(int)));
    int *nodeOutputResult;
    nodeOutputResult = (int*)malloc(numNodes*sizeof(int)); 
    gpuErrorCheck(cudaMemcpy(nodeOutputResult, nodeOutput_d, numNodes*sizeof(int), cudaMemcpyDeviceToHost));

    int *nextLevelNodes_h; // to write in output file
    nextLevelNodes_h = ( int *)malloc(numNextLevelNodes_h * sizeof(int));

    gpuErrorCheck(cudaMemcpy(nextLevelNodes_h, nextLevelNodes_d, numNextLevelNodes_h * sizeof(int), cudaMemcpyDeviceToHost));

    FILE* output_file_node = fopen(output_nodeOutput, "w");
    FILE* output_file_nextLevelNode = fopen(output_nextLevelNodes, "w");

    // Print out the results
    fprintf(output_file_node, "%d\n", numNodes);
    for (int i = 0; i < numNodes; i++) { 
        fprintf(output_file_node, "%d\n", nodeOutputResult[i]); 
    }
    fclose(output_file_node);

    fprintf(output_file_nextLevelNode, "%d\n", numNextLevelNodes_h);
    for (int i = 0; i < numNextLevelNodes_h; i++) { 
        fprintf(output_file_nextLevelNode, "%d\n", nextLevelNodes_h[i]); 
    }
    fclose(output_file_nextLevelNode);
}

Writing global_queuing.cu


## Compile and Run

In [ ]:
!nvcc global_queuing.cu -o global_queuing

global_queuing.cu(139): warning: variable "debug" was declared but never referenced

global_queuing.cu(213): warning: variable "numTotalNeighbors_d" was declared but never referenced



### 10 blocks, 32 threads per block 

In [ ]:
!./global_queuing 10 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

10 Blocks, 32 Threads/Block
32 Nodes per Thread
*** Global Queuing Execution Time: 0.576576 ms***


### 25 blocks, 32 threads per block 

In [ ]:
!./global_queuing 25 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 32 Threads/Block
13 Nodes per Thread
*** Global Queuing Execution Time: 0.243904 ms***


### 35 blocks, 32 threads per block 

In [ ]:
!./global_queuing 35 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 32 Threads/Block
9 Nodes per Thread
*** Global Queuing Execution Time: 0.174528 ms***


### 10 blocks, 64 threads per block 

In [ ]:
!./global_queuing 10 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

10 Blocks, 64 Threads/Block
16 Nodes per Thread
*** Global Queuing Execution Time: 0.300896 ms***


### 25 blocks, 64 threads per block 

In [ ]:
!./global_queuing 25 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 64 Threads/Block
7 Nodes per Thread
*** Global Queuing Execution Time: 0.149664 ms***


### 35 blocks, 64 threads per block 

In [ ]:
!./global_queuing 35 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 64 Threads/Block
5 Nodes per Thread
*** Global Queuing Execution Time: 0.113728 ms***


### 10 blocks, 128 threads per block 

In [ ]:
!./global_queuing 10 128 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

10 Blocks, 128 Threads/Block
8 Nodes per Thread
*** Global Queuing Execution Time: 0.170688 ms***


### 25 blocks, 128 threads per block 

In [ ]:
!./global_queuing 25 128 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 128 Threads/Block
4 Nodes per Thread
*** Global Queuing Execution Time: 0.097024 ms***


### 35 blocks, 128 threads per block 

In [ ]:
!./global_queuing 35 128 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 128 Threads/Block
3 Nodes per Thread
*** Global Queuing Execution Time: 0.075616 ms***


### Verify Results

In [ ]:
!./compareNextLevelNodes sol_nextLevelNodes.raw nextLevelNodes.raw

No errors!


In [ ]:
!./compareNodeOutput sol_nodeOutput.raw nodeOutput.raw

Total Errors : 0	

# Parallel BFS (block-level queues)

In [ ]:
%%writefile block_queuing.cu
#include <stdio.h>
#include <stdlib.h>
// CUDA runtime
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

#define MAX_QUEUE_SIZE 200000
#define MAX_BLOCK_QUEUE_SIZE 2000

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

__device__ int numNextLevelNodes_d = 0;

// Macro for Cuda Debugging
#define gpuErrorCheck(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}


__device__ int gate_solver(int gate, int input1, int input2) {
    int output;
    switch (gate) {
            case AND:
                output = input1 && input2;
                break;
            case OR:
                output = input1 || input2;
                break;
            case NAND:
                output = !(input1 && input2);
                break;
            case NOR:
                output = !(input1 || input2);
                break;
            case XOR:
                output = (input1 || input2) && !(input1 && input2);
                break;
            case XNOR:
                output = (input1 && input2) || (!input1 && !input2);
                break;
    }
    return output;
}

/**
 * @brief Reads an input file containing length L in first line and one integer for each of the next L lines, storing the values in the integer array provided.
 * 
 * @param input1 pointer to an integer array to store the data
 * @param filepath character array, name of input file to open
 * @return int, length of resulting array
 */
int read_input_one_two_four(int **input1, char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    
    // Allocate len entries and assign the pointer to out input reference
    *input1 = ( int *)malloc(len * sizeof(int));
    int temp1;
    
    // read line by line, one integer per line
    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;

}

/**
 * @brief Reads a CSV formatted file with information for every node gate.
 * 
 * @param input1 Pointer to Visited array
 * @param input2 Pointer to Logic Gate types array
 * @param input3 Pointer to Logic Gate inputs array
 * @param input4 Pointer to Logic Gate outputs array
 * @param filepath 
 * @return int, the number of node entries
 */
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));    // visited[i], either 0 or 1
    *input2 = ( int *)malloc(len * sizeof(int));    // gate[i], see macros definitions
    *input3 = ( int *)malloc(len * sizeof(int));    // input[i], fixed input to node i
    *input4 = ( int *)malloc(len * sizeof(int));    // output[i], -1 if not computed

    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len; // number of total nodes in circuit
}


__global__ void blockLevelQueueBFS(int* currLevelNodes, int numCurrLevelNodes, int* nodePtrs, int* nodeNeighbors, int* nodeVisited, int* nodeGate, int* nodeInput, int* nodeOutput, int* nextLevelNodes, int nodes_per_thread, int BLOCK_QUEUE_SIZE) {
    
    __shared__ unsigned int block_queue[MAX_BLOCK_QUEUE_SIZE];
    __shared__ unsigned int blockQueueTail,  globalQueueOffset; // rename

    if (threadIdx.x == 0) {
        blockQueueTail = 0;
    }
    __syncthreads();    // do not proceed until all threads have reached this point.

    const unsigned int tid = (blockIdx.x * blockDim.x) + threadIdx.x;

    const unsigned int start = tid * nodes_per_thread;
    const int thread_limit = start + nodes_per_thread;

    for (unsigned int idx = start; idx < thread_limit && idx < numCurrLevelNodes; idx++) {
        // get the node
        unsigned int node = currLevelNodes[idx];

        // Loop through neighbors
        for (unsigned int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) {
            int neighbor = nodeNeighbors[nbrIdx];
            // If neighbor hasn't been visited yet
            int already_visited = atomicExch(&(nodeVisited[neighbor]), 1);
            if (!already_visited) {
                int output = gate_solver(nodeGate[neighbor], nodeInput[neighbor], nodeOutput[node]);
                nodeOutput[neighbor] = output;
                int new_tail = atomicAdd(&blockQueueTail, 1); // check
                
                if (new_tail < BLOCK_QUEUE_SIZE) {
                    // Add to block level queue
                    block_queue[new_tail] = neighbor;
                } else {
                    blockQueueTail = BLOCK_QUEUE_SIZE;
                    // Atomically add to Global queue
                    int new_i = atomicAdd(&numNextLevelNodes_d, 1); // check
                    nextLevelNodes[new_i] = neighbor;
                }
            }
        }
    }

    __syncthreads();
    if (threadIdx.x == 0) {
        globalQueueOffset = atomicAdd(&numNextLevelNodes_d, blockQueueTail);
    }

    __syncthreads();
    // Copy our block's queue into the global queue, splitting work evenly among blockDim.x threads
    for (unsigned int i = threadIdx.x; i < blockQueueTail; i += blockDim.x) {
            nextLevelNodes[globalQueueOffset + i] = block_queue[i];
    }

}


int main(int argc, char *argv[]) {
    // Variables
    if (argc != 10)
    {
        printf("\nIncorrect number of arguments. Please run the program as follows:\n");
        printf("./block_queuing <numBlock> <blockSize> <sharedQueueSize> <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    }
    // Read input and output file names from command
    int numBlock;
    int blockSize;
    int sharedQueueSize; 
    sscanf(argv[1], "%d", &numBlock);
    sscanf(argv[2], "%d", &blockSize);
    sscanf(argv[3], "%d", &sharedQueueSize);

    char* input_1 = argv[4]; 
    char* input_2 = argv[5]; 
    char* input_3 = argv[6];
    char* input_4 = argv[7];
    char* output_nodeOutput = argv[8]; 
    char* output_nextLevelNodes = argv[9];
    
    // Variables
    int numNodePtrs;          
    int numNodes;             
    int *nodePtrs_h;          
    int *nodeNeighbors_h;      
    int *nodeVisited_h;   
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes;
    int numNextLevelNodes_h;
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;
    
    numNextLevelNodes_h = 0;
    numNodePtrs = read_input_one_two_four(&nodePtrs_h, input_1);
    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, input_2);
    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);
    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);
    
    // Allocate GPU Memory 

    int *nodePtrs_d;          
    int *nodeNeighbors_d;      
    int *nodeVisited_d;   
    int numTotalNeighbors_d;
    int *currLevelNodes_d;
    int *nodeGate_d;
    int *nodeInput_d;
    int *nodeOutput_d;

    int* nextLevelNodes_d;
    gpuErrorCheck(cudaMalloc((void **)&nextLevelNodes_d, MAX_QUEUE_SIZE * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeVisited_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodePtrs_d, numNodePtrs * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeNeighbors_d, numTotalNeighbors_h * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&currLevelNodes_d, numCurrLevelNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeGate_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeInput_d, numNodes * sizeof(int)));
    gpuErrorCheck(cudaMalloc((void **)&nodeOutput_d, numNodes * sizeof(int)));

    gpuErrorCheck(cudaMemcpy(nodeVisited_d, nodeVisited_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodePtrs_d, nodePtrs_h, numNodePtrs * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeNeighbors_d, nodeNeighbors_h, numTotalNeighbors_h * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(currLevelNodes_d, currLevelNodes_h, numCurrLevelNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeGate_d, nodeGate_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeInput_d, nodeInput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));
    gpuErrorCheck(cudaMemcpy(nodeOutput_d, nodeOutput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice));


    // Start timer
    float memsettime;
    cudaEvent_t start, stop;
    gpuErrorCheck(cudaEventCreate(&start));
    gpuErrorCheck(cudaEventCreate(&stop));
    gpuErrorCheck(cudaEventRecord(start, 0));

    // CALL THE KERNEL
    printf("%d Blocks, %d Threads/Block, %d Shared Queue Size\n", numBlock, blockSize, sharedQueueSize);

    int capacity = numBlock * blockSize;
    int remainder = (int)((numCurrLevelNodes % capacity) > 0);
    int nodes_per_thread = (numCurrLevelNodes / capacity) + remainder;
    printf("%d Nodes per Thread\n", nodes_per_thread);
    blockLevelQueueBFS<<<numBlock, blockSize>>>(currLevelNodes_d, numCurrLevelNodes, nodePtrs_d, nodeNeighbors_d, nodeVisited_d, nodeGate_d, nodeInput_d, nodeOutput_d, nextLevelNodes_d, nodes_per_thread, sharedQueueSize);
    cudaDeviceSynchronize();

    // Stop timer
    gpuErrorCheck(cudaEventRecord(stop, 0));
    gpuErrorCheck(cudaEventSynchronize(stop));
    gpuErrorCheck(cudaEventElapsedTime(&memsettime, start, stop));
    // print total time elapsed for global queuing gate simulation
    printf("*** Block Queuing Execution Time: %f ms ***\n", memsettime);

    gpuErrorCheck(cudaEventDestroy(start));
    gpuErrorCheck(cudaEventDestroy(stop));

    // Cleanup;
    cudaFree(nodePtrs_d);
    free(nodePtrs_h);
    cudaFree(nodeNeighbors_d);
    free(nodeNeighbors_h);
    cudaFree(nodeVisited_d);
    free(nodeVisited_h);
    cudaFree(currLevelNodes_d);
    free(currLevelNodes_h);
    cudaFree(nodeGate_d);
    free(nodeGate_h);
    cudaFree(nodeInput_d);
    free(nodeInput_h);

    // Copy results
    gpuErrorCheck(cudaMemcpyFromSymbol(&numNextLevelNodes_h, numNextLevelNodes_d, sizeof(int)));
    int *nodeOutputResult;
    nodeOutputResult = (int*)malloc(numNodes*sizeof(int)); 
    gpuErrorCheck(cudaMemcpy(nodeOutputResult, nodeOutput_d, numNodes*sizeof(int), cudaMemcpyDeviceToHost));

    int *nextLevelNodes_h; // to write in output file
    nextLevelNodes_h = ( int *)malloc(numNextLevelNodes_h * sizeof(int));

    gpuErrorCheck(cudaMemcpy(nextLevelNodes_h, nextLevelNodes_d, numNextLevelNodes_h * sizeof(int), cudaMemcpyDeviceToHost));

    FILE* output_file_node = fopen(output_nodeOutput, "w");
    FILE* output_file_nextLevelNode = fopen(output_nextLevelNodes, "w");

    // Print out the results
    fprintf(output_file_node, "%d\n", numNodes);
    for (int i = 0; i < numNodes; i++) { 
        fprintf(output_file_node, "%d\n", nodeOutputResult[i]); 
    }
    fclose(output_file_node);

    fprintf(output_file_nextLevelNode, "%d\n", numNextLevelNodes_h);
    for (int i = 0; i < numNextLevelNodes_h; i++) { 
        fprintf(output_file_nextLevelNode, "%d\n", nextLevelNodes_h[i]); 
    }
    fclose(output_file_nextLevelNode);

    // Lastly...
    free(nextLevelNodes_h);
    cudaFree(nextLevelNodes_d);
    free(nodeOutputResult);
    cudaFree(nodeOutput_d);
}

Writing block_queuing.cu


## Compile and Run

In [ ]:
!nvcc block_queuing.cu -o block_queuing

block_queuing.cu(235): warning: variable "numTotalNeighbors_d" was declared but never referenced



### 25 blocks, 32 threads per block, 32 shared queue size

In [ ]:
!./block_queuing 25 32 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 32 Threads/Block, 32 Shared Queue Size
13 Nodes per Thread
*** Block Queuing Execution Time: 0.261600 ms ***


### 25 blocks, 32 threads per block, 64 shared queue size 

In [ ]:
!./block_queuing 25 32 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 32 Threads/Block, 64 Shared Queue Size
13 Nodes per Thread
*** Block Queuing Execution Time: 0.262912 ms ***


### 35 blocks, 32 threads per block, 32 shared queue size

In [ ]:
!./block_queuing 35 32 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 32 Threads/Block, 32 Shared Queue Size
9 Nodes per Thread
*** Block Queuing Execution Time: 0.197728 ms ***


### 35 blocks, 32 threads per block, 64 shared queue size

In [ ]:
!./block_queuing 35 32 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 32 Threads/Block, 64 Shared Queue Size
9 Nodes per Thread
*** Block Queuing Execution Time: 0.188352 ms ***


### 25 blocks, 64 threads per block, 32 shared queue size

In [ ]:
!./block_queuing 25 64 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 64 Threads/Block, 32 Shared Queue Size
7 Nodes per Thread
*** Block Queuing Execution Time: 0.158560 ms ***


### 25 blocks, 64 threads per block, 64 shared queue size

In [ ]:
!./block_queuing 25 64 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

25 Blocks, 64 Threads/Block, 64 Shared Queue Size
7 Nodes per Thread
*** Block Queuing Execution Time: 0.153984 ms ***


### 35 blocks, 64 threads per block, 32 shared queue size

In [ ]:
!./block_queuing 35 64 32 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 64 Threads/Block, 32 Shared Queue Size
5 Nodes per Thread
*** Block Queuing Execution Time: 0.118624 ms ***


### 35 blocks, 64 threads per block, 64 shared queue size

In [ ]:
!./block_queuing 35 64 64 input1.raw input2.raw input3.raw input4.raw nodeOutput.raw nextLevelNodes.raw

35 Blocks, 64 Threads/Block, 64 Shared Queue Size
5 Nodes per Thread
*** Block Queuing Execution Time: 0.114272 ms ***


### Verify Results

In [ ]:
!./compareNextLevelNodes sol_nextLevelNodes.raw nextLevelNodes.raw

No errors!


In [ ]:
!./compareNodeOutput sol_nodeOutput.raw nodeOutput.raw

Total Errors : 0	